# After years of thought it's finally here

In [3]:
import pandas as pd 
import numpy as np
pd.set_option('display.max_rows', 10)

In [4]:
df=pd.read_excel('Pokemon.xlsx')

ditto has been removed from the list

mons with a mega evolution with a new type, have the new type as the 'third type'

mons with an ability like Steelworker or Rocky Payload have their third type as the 1.5 boosted type
(this is for the sake of stab being appreciated against elite types)
However the moves of this third type will still be noted down

#moves that already share stab with the mon are not included, this program is just to find coverage

#the moves included are moves above a power of 65 and above

#except bulldoze, rock tomb, low sweep are included, breaking swipe is included (chilling water, flame charge and trailblaze are not included) (icy wind and electroweb also not included)

#if a mon gets flamethrower, I won't write fire blast or heat wave(welp I didn't follow this properly but whatever), and if a mon gets thunderbolt, I won't write thunder, no blizzard if it learns ice beam

#priority moves with low BP have been included, also pivot moves with low bp like flip turn (haven't included normal type priority though)

#multi hit moves will be included (even beat up)

#stored power, avalanche, fling, pursuit, payback, acrobatics, revenge and reversal are also included

#stomping tantrum and high horsepower will be avoided if it already learns earthquake

#superpower will be avoided if it learns close combat

#U-turn is spelled as uturn, X-Scissor is X Scissor

#for mons with technician I'll add moves above 40

#no dream eater, no upper hand, no focus punch, no psywave, no magnitude, no steel roller
#no recharge moves like hyper beam, rock wrecker

#yes two turn moves

#mirror coat, metal burst and counter are typeless

#not including natural gift which gives unlimited coverage


In [5]:
display=pd.concat([df.loc[0:25,'Name':'MonType3'],df.loc[0:25,'Move1':]],axis=1)
display
#this was me trying to display a small part (25 rows) of the dataframe to see if it loaded correctly.
# I excluded columns like RU, Used, for clarity.


,Name,MonType1,MonType2,MonType3,Move1,Type1,Move2,Type2,Move3,Type3,...,Move38.1,Type39,Move40,Type40,Move41,Type41,Move42,Type42,Move43,Type43
0,Abomasnow,Grass,Ice,NaN,Brick Break,Fighting,Earthquake,Ground,Earth Power,Ground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Absol,Dark,NaN,NaN,Close Combat,Fighting,Iron Tail,Steel,Megahorn,Bug,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Accelgor,Bug,NaN,NaN,Focus Blast,Fighting,Energy Ball,Grass,Sludge Bomb,Poison,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Aerodactyl,Rock,Flying,NaN,Aqua Tail,Water,Crunch,Dark,Dragon Claw,Dragon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Aggron,Steel,Rock,NaN,Aqua Tail,Water,Body Press,Fighting,Avalanche,Ice,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21,Armarouge,Fire,Psychic,NaN,Aura Sphere,Fighting,Dark Pulse,Dark,Dragon Pulse,Dragon,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22,Aromatisse,Fairy,NaN,NaN,Energy Ball,Grass,Flash Cannon,Steel,Psychic,Psychic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23,Articuno,Ice,Flying,NaN,Extrasensory,Psychic,Freeze Dry,Electric,Signal Beam,Bug,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
24,Articuno-Galar,Psychic,Flying,NaN,Shadow Ball,Ghost,Uturn,Bug,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
'''df.columns=df.columns.str.replace('Coverage','Type')
df.columns
df.to_excel('Pokemon.xlsx',index=False)'''
#Initially the excel file had 'Coverage1','Coverage2' as column names, I replaced them with 'Type1','Type2' permanently.
# So now they always load as Type1, Type2.

"df.columns=df.columns.str.replace('Coverage','Type')\ndf.columns\ndf.to_excel('Pokemon.xlsx',index=False)"

In [7]:
#Type Offense dictionary
#mirror coat and counter are typeless
Off={
    'Typeless':[],
    'Normal':[],
    'Fire':['Grass','Ice','Bug','Steel'],
    'Water':['Fire','Ground','Rock'],
    'Grass':['Water','Ground','Rock'],
    'Electric':['Water','Flying'],
    'Ice':['Grass','Ground','Flying','Dragon'],
    'Fighting':['Normal','Ice','Rock','Dark','Steel'],
    'Poison':['Grass','Fairy'],
    'Ground':['Fire','Electric','Poison','Rock','Steel','Dragon'],    #ground is good against nearly half dragons so I added that
    'Flying':['Grass','Fighting','Bug'],
    'Psychic':['Fighting','Poison'],
    'Bug':['Grass','Psychic','Dark'],
    'Rock':['Fire','Ice','Flying','Bug','Dragon'],   #rock is good against nearly half dragons so I added that
    'Ghost':['Psychic','Ghost'],
    'Dragon':['Dragon'],
    'Dark':['Psychic','Ghost'],
    'Steel':['Ice','Rock','Fairy'],
    'Fairy':['Fighting','Dragon','Dark'],
    'nan' :[]
}
#notes: freeze dry is sneakily seen as an electric type move to include super effectiveness on water

In [8]:
offdf=pd.Series(Off) #DataFrame doesn't work because all arrays need to be the same length, Series allows variable length arrays

In [9]:
offdf #this has the indexes as the types, so it's easy to access the values with loc

Typeless                          []
Normal                            []
Fire        [Grass, Ice, Bug, Steel]
Water           [Fire, Ground, Rock]
Grass          [Water, Ground, Rock]
                      ...           
Dragon                      [Dragon]
Dark                [Psychic, Ghost]
Steel             [Ice, Rock, Fairy]
Fairy       [Fighting, Dragon, Dark]
nan                               []
Length: 20, dtype: object

In [18]:
'''
moves_columns=df.columns[df.columns.str.contains('Move', na=False)].tolist()
moves_columns
moves_columns.append('Name') #I append Name, but later I set this as the index
''' #These lines do the same thing as the rest of the code in this cell.

#moves_columns=df.columns[df.columns.str.contains('Move', na=False)].append(pd.Index(['Name']))
#I didn't have to convert to list and back to Index, pandas allows appending Index objects (even this is lame though)
#The thing inside the brackets is a boolean mask that filters columns with 'Move' in their name. So it has true false values.
#After converting the true values into the actual column names, I appended the Name column.
#movesdf=df[moves_columns]
# this unnecessarily uses .str functions, I didn't know that column names were already inside a list
# and that it's dumb to create complicated filters (boolean masks) for just a list not a dataframe.

# Like you can see the messiness of 'df.columns[df.columns', this later being furtherinside df[] before being assigned to movesdf.
# My list comprehension is cleaner.

'''
#movesdf=movesdf.dropna() #this is bad, this drops all incomplete rows, and hence the only row left is the longest row, Rhyperior's row, which has 43 moves.

# Let's just keep the nulls values the way they are

'''
movesdf=df[   ['Name'] + [x for x in df.columns if x.startswith('Move')]   ]


movesdf=movesdf.set_index('Name')
movesdf


,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,...,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,Fling,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,Pursuit,Drain Punch,Reversal,Venoshock,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,Pursuit,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Counter,Reversal,Fling,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,...,Rock Tomb,Payback,Pursuit,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Close Combat,Low Kick,Hammer Arm,Iron Tail,Dig,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#type_columns=df.columns[df.columns.str.contains('Type', na=False)].tolist() 
#type_columns
#type_columns.append('Name')
#type_df=df[type_columns]
#commenting cuz it's messy like the above cell
#searching for columns with the word 'type' works out well cuz you get both the Montype columns and nonstab type columns
type_df=df[  ['Name'] + [x for x in df.columns if 'Type' in x ]  ]   #This is way cleaner than .str.contains
#It's crazy that python has a built-in function to find substrings.

type_df=type_df.set_index('Name')
type_df

,MonType1,MonType2,MonType3,Type1,Type2,Type3,Type4,Type5,Type6,Type7,...,Type34,Type35,Type36,Type37,Type38,Type39,Type40,Type41,Type42,Type43
Name,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,Fighting,Ground,Ground,Steel,Fighting,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,Fighting,Steel,Bug,Fairy,Psychic,Rock,Ghost,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,Fighting,Grass,Poison,Water,Dark,Dark,Fighting,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,Water,Dark,Dragon,Ground,Fire,Ice,Steel,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,Water,Fighting,Ice,Dark,Dragon,Ground,Fire,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,Fighting,Grass,Psychic,Dark,Ghost,Grass,Psychic,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,Typeless,Fighting,Dark,Fighting,Fighting,Grass,Poison,...,Rock,Dark,Dark,Dark,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,Fighting,Fighting,Fighting,Steel,Ground,Rock,Ground,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Code to find where certain moves (or types) have been misspelled a particular way

In [ ]:
#Temporary code, apparently two of the moves are named 'Countr', which means I messed up somwehere during input,
# so I'm trying to find the index of the place I messed up so that I can edit it in Excel
#once I fix it, this code will be useless. Only useable for finding future errors.

indices=np.where(movesdf=='Countr')

#print(indices)

row_indices, col_indices = indices[0], indices[1]
print(row_indices,col_indices)

# indices start from 0, and moreover, one row is used up to write the names of the columns
# so an x value of 104 actually corresponds to row 106 in excel.


print(movesdf.iloc[102,1]) #just prints the  cell with the misspelling again
#the above line is the hardcode, the general code would be:
if len(row_indices)*len(col_indices)>0: #they have to exist otherwise the following line will error due to empty lists
    print(movesdf.iloc[row_indices[0],col_indices[0]]) 



for x,y in zip(row_indices, col_indices):
    print(movesdf.index[x],movesdf.columns[y])  #prints the position of the errors in terms of pokemon name and move column


'''
old code where I manually read the output, found the indexes, and plugged them in to see the location.

print(df.loc[32,'Move12'])
print(df.loc[32,'Move13'])

print(df.loc[32,'Name'])
'''

[] []
Counter


"\nold code where I manually read the output, found the indexes, and plugged them in to see the location.\n\nprint(df.loc[32,'Move12'])\nprint(df.loc[32,'Move13'])\n\nprint(df.loc[32,'Name'])\n"

In [ ]:
#exploration code

#set(np.append((np.append(display['Move1'].unique(),display['Move2'].unique())),display['Move3'].unique()))
#set(display['Move1'].tolist()+display['Move2'].tolist()+display['Move3'].tolist())
# This neste append is so trash. Back then I didn't know that you can append multiple arrays at once.
# Writing out the column names (Move1, Move2, Move3) is also so trash.
# set() is inferior to drop_duplicates()




#when I combined different move columns into one series, I got repeated indexes, which were weirdly nested.

#moveslist=pd.concat([df['Move1'],df['Move2'],df['Move3']]).reset_index() #without this, every index has values of all the different moves
# but after resetting index, every move gets numbers as indexes. And the moves have multiple numerical indexes, they're nested indexes.
#moveslist.columns=['old_repeated_index','moves']
#moveslist.dropna()
#moveslist[moveslist['old_repeated_index']==2]
#moveslist=moveslist.drop('old_repeated_index',axis=1)

# the above block of code was shown to be a waste when I discovered ignore_index=True


All the move columns are merged into one column, and then duplicates are dropped to get a proper list of moves

In [ ]:
#moveslist=pd.concat([df['Move1'],df['Move2'],df['Move3'],df['Move4'],df['Move5'],df['Move6'],df['Move7'],df['Move8'],df['Move9'],df['Move10'],df['Move11'],df['Move12'],df['Move13'],df['Move14'],df['Move15'],df['Move16'],df['Move17'],df['Move18'],df['Move19'],df['Move20'],df['Move21'],df['Move22'],df['Move23'],df['Move24'],df['Move25'],df['Move26'],df['Move27'],df['Move28'],df['Move29'],df['Move30'],df['Move31'],df['Move32'],df['Move33'],df['Move34'],df['Move35'],df['Move36']], ignore_index=True)
# straight up bad code


#moveslist=pd.concat([col for col in movesdf], axis = 0, ignore_index=True)
#this line of code doesn't work because col is just the name of the column, not the contents of the column.
#it turns out that if you try to iterate an element over a dataframe, you just get the column names.
#so you have to do movesdf[col] for col in movesdf.


moveslist=pd.concat([movesdf[col] for col in movesdf.columns], axis = 0, ignore_index=True)
#concat selects columns one by one and concatenates their contents vertically

print(len(moveslist))

moveslist=moveslist.dropna() # the only way columns wouldn't have empty cells is if every pokemon had 43 moves


print(len(moveslist)) #this is 20984 before dropping empty cells, and 6728 after dropping. (20984 is 488 pokemon * 43 max moves)
print(moveslist.nunique()) #this is 213 (there are 213 unique moves)

20984
6728
212


In [ ]:
moveslist.head()
#moveslist is a series and that's why the output is displayed like this

0     Brick Break
1    Close Combat
2     Focus Blast
3       Aqua Tail
4       Aqua Tail
dtype: object

create a filter later so that moves that rely on the significantly weaker stat can be removed if desired

so then that means we create a column for a pokemon, which states its dominant offense, and if no dominant offense then it's empty

my latest idea on this is to add the physical and special attack of every mon, and sort based on this. 

remember to somehow let counter and mirror coat and metal burst maybe remain unfiltered as strong moves

maybe we should remember when even special attackers have moves like uturn, or physical attackers have moves like volt switch
Both of these should count as supereffective good moves because the BP exceeds 60.

I should actually create a whole ass column called pivot which shows when a mon has a pivot move or not, and what this/these priority move is/are
This requires me to type in uturn for a bunch of bugs too. (and volt switch/flip turn). Also need to code in parting shot, a non offensive move.

and priority moves also should be available to both, cuz the point is usually just to break sash
you know what, maybe I should make a column called priority and add in all the normal priority moves like quick attack too.
I probably won't add grassy glide, but I'll force add Comfey's draining moves as priority particulary for it.

 do all these complicated actions with df.query

one filter should also make sure that a type is stab supereffective for a mon (but also remember that freeze dry is electric type and needs to be counted as stab)

vectorized method usually means you create another column, like df['result']=df[first] + df[second]

Also don't heavily deprioritize solar beam and stuff, cuz z moves are a thing. Maybe mention it in your application that the recommended strategy is to z move


have a filter to avoid inconsistent moves like fling, pursuit
maybe bulldoze and rock tomb too, although those moves could come under another filter

Remember to code something later that finds out how many types of moves a mon has (not coverage they apply on, but simply how many types of moves it has. The mons with less types of moves are harder to use in the gauntlet eventually, they'll be rarely picked by the code. So in the chance that they are picked by the code, highly prioritize using  them (like articuno). So try to sort them super high)


You can call this, "Niche pokemon for whom these 4 types are the perfect niche"


In [ ]:
'''
indices=np.where(type_df=="Earthquake") #this mistaken spot where the type of bulldoze was earthquake
row_indices, col_indices = indices[0], indices[1]

print(type_df.iloc[row_indices[0],col_indices[0]]) # this just prints the Earthquake again to double check.

print(int(col_indices[0])) #this is the column index in typedf. There are 3 Montype columns (which start from index 0), so Type1 is index 3, Type2 is index 4

print(type_columns[int(col_indices[0])]) #This inputs the column index into type_columns list to get the column name, e.g. Type12

#From here on, I think I tried to explore if I could link the type of a move back to the move itself.

print(type_columns[int(col_indices[0])][4:]) #This slices the previous result (truncating the letters T-Y-P-E) to get just the number, e.g. 12.

col_num=type_columns[int(col_indices[0])][4:] #cutting down Type12 into 12
print('Move'+ col_num) #so we're able to go a cell back and access the move manually


'''
# I think I was trying to link the type of a move back to the move itself. 
# This is dogshit code, dumb me of the past sliced the letters of Type and added the letters of Move,
# by converting the index of the type_df to the column name, and then to the column name of the moves_df.
# This was unnecessary, I should have just stayed in index for both dataframes. It is true that type_df is a little wider than moves_df, because there are 3 stab types, but I could have subtracted easily.
# Young me was too dumb to think in terms of indexes, and that's why he tried string bs.


'\nindices=np.where(type_df=="Earthquake") #this mistaken spot where the type of bulldoze was earthquake\nrow_indices, col_indices = indices[0], indices[1]\n\nprint(type_df.iloc[row_indices[0],col_indices[0]]) # this just prints the Earthquake again to double check.\n\nprint(int(col_indices[0])) #this is the column index in typedf. There are 3 Montype columns (which start from index 0), so Type1 is index 3, Type2 is index 4\n\nprint(type_columns[int(col_indices[0])]) #This inputs the column index into type_columns list to get the column name, e.g. Type12\n\n#From here on, I think I tried to explore if I could link the type of a move back to the move itself.\n\nprint(type_columns[int(col_indices[0])][4:]) #This slices the previous result (truncating the letters T-Y-P-E) to get just the number, e.g. 12.\n\ncol_num=type_columns[int(col_indices[0])][4:] #cutting down Type12 into 12\nprint(\'Move\'+ col_num) #so we\'re able to go a cell back and access the move manually\n\n\n'

#### A way to hunt down where there's a mismatch between the move name and the type it's given:
Shift this code (above and below) to have a proper group of 'error in data.'

In [ ]:
# This prints all the indices where Counter is found. And the indices where Typeless is found.
# Maybe I can comment this code now instead of letting it run every time.

indices=np.where(movesdf=="Counter")
row_indices_m, col_indices_m = indices[0], indices[1]
print(col_indices_m)

indices=np.where(type_df=="Typeless")
row_indices_t, col_indices_t = indices[0], indices[1]
print(col_indices_t)

#There is an inequality between the number of times Counter is found and the number of times Typeless is found. 
#So make sure to use both indexes.

print('')


# I'm trying to find out where Counter is found and the corresponding type is not typeless.
# Because my list tells me that there's one particular Counter which claims to be fighting type. (I know I could do == fighting instead of != typeless, but whatever)
# Since I solved the issue, the following code no longer has any output.

for x,y in zip(row_indices_m, col_indices_m):
    if type_df.iloc[x,y+3] != "Typeless":  # +3 because type df has 3 extra columns at the start for Montype1, Montype2, Montype3
        print(x,y+3)                       # so traversing between type df and moves df requires subtracting or adding 3 columns 

        print(type_df.index[x],type_df.columns[y+3])  #prints the position of the non-typeless counter in terms of pokemon name and type column

# Most of these are 0 because when the most common index for Counter to be found in movesdf is index 0.
# Same goes for typeless moves in type_df
# subtracting by 3 is the right way to traverse between type df and moves df, because type df has 3 extra columns at the start for Montype1, Montype2, 



[15 27 20  7  8 14  9  9  8  8  1  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0 22  0 14 15  0  0  0  0  0  0  0  0  0  0  0  0  0
 19  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0 11 40 24 39  2  0
  0  0  7  0  0  0  0  0  0  0  0  0  0  0  0]
[18 30 23 10  7  6  7 15  9 11 17 17 18 12 12 11 11 12  3  7 11 13  4  3
  4  3  3  3  3  3 15  3  3  3  3  3  3  3  3  3  3  3  3  4  4  3  3  3
  3  3  3  3  3  4 25  3 17 18  3  3  3  4  3  3  3  3  3  3  3  3  3  3
  3  3 22  3  3  3  3  3  3  3  3  3  3  4  3  3  3  3  3  3  3  4  3  3
  3  3  3  3  3  3  3  3  3  3  3  3  3  3  3  4  3  3  3  3  3  3  3  3
  3  3  4  3  3  4  5  5  3  3  3 14 43 27 42  5  3  3  3  3 10  3  4  3
  3  4  3  3  3  3  3  3  4  3  3  3  4  3  3  3]



In [ ]:
pd.set_option('display.max_columns', 50)

In [ ]:
# This code is absolutely faulty


def from_type_to_moves(type):
    indices=np.where(type_df==type)         #all the cells in the database where you find the desired type
    row_indices, col_indices = indices[0], indices[1]
    col_indices_moves=col_indices-3

    print(len(col_indices_moves))
    print(len(np.unique(col_indices_moves)))
    #There are only 21 unique columns out of 43 which have the type 'typeless', but they get repeated up to 160 times.

    #print((type_df.iloc[row_indices]).index) 
    return movesdf.iloc[row_indices,col_indices_moves]   
    #This code is straight up faulty.
    #If the type 'typeless' was found at cells 1,3 and 9,6 then this code prints the entire row 1, row 9, column 3 and column 6.



#from_type_to_moves('Typeless')
#This calls the function, but it's a useless function.

160
21


,Move16,Move28,Move21,Move8,Move5,Move4,Move5,Move13,Move7,Move9,Move15,Move15,Move16,Move10,Move10,Move9,Move9,Move10,Move1,Move5,Move9,Move11,Move2,Move1,Move2,...,Move3,Move1,Move1,Move1,Move1,Move8,Move1,Move2,Move1,Move1,Move2,Move1,Move1,Move1,Move1,Move1,Move1,Move2,Move1,Move1,Move1,Move2,Move1,Move1,Move1
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Absol,Counter,NaN,NaN,Stone Edge,Psycho Cut,Play Rough,Psycho Cut,Shadow Ball,Shadow Claw,X Scissor,Future Sight,Future Sight,Counter,Zen Headbutt,Zen Headbutt,X Scissor,X Scissor,Zen Headbutt,Close Combat,Psycho Cut,X Scissor,Bounce,Iron Tail,Close Combat,Iron Tail,...,Megahorn,Close Combat,Close Combat,Close Combat,Close Combat,Stone Edge,Close Combat,Iron Tail,Close Combat,Close Combat,Iron Tail,Close Combat,Close Combat,Close Combat,Close Combat,Close Combat,Close Combat,Iron Tail,Close Combat,Close Combat,Close Combat,Iron Tail,Close Combat,Close Combat,Close Combat
Aggron,Dark Pulse,Counter,Thunderbolt,Ice Punch,Dragon Claw,Crunch,Dragon Claw,Thunder Punch,Fire Punch,Low Kick,Dragon Pulse,Dragon Pulse,Dark Pulse,Outrage,Outrage,Low Kick,Low Kick,Outrage,Aqua Tail,Dragon Claw,Low Kick,Shadow Claw,Body Press,Aqua Tail,Body Press,...,Avalanche,Aqua Tail,Aqua Tail,Aqua Tail,Aqua Tail,Ice Punch,Aqua Tail,Body Press,Aqua Tail,Aqua Tail,Body Press,Aqua Tail,Aqua Tail,Aqua Tail,Aqua Tail,Aqua Tail,Aqua Tail,Body Press,Aqua Tail,Aqua Tail,Aqua Tail,Body Press,Aqua Tail,Aqua Tail,Aqua Tail
Ambipom,Trailblaze,Fling,Counter,Seed Bomb,Gunk Shot,Foul Play,Gunk Shot,Dual Chop,Knock Off,Shadow Claw,Fury Cutter,Fury Cutter,Trailblaze,Thunder Punch,Thunder Punch,Shadow Claw,Shadow Claw,Thunder Punch,Acrobatics,Gunk Shot,Shadow Claw,Triple Axel,Low Kick,Acrobatics,Low Kick,...,Fire Punch,Acrobatics,Acrobatics,Acrobatics,Acrobatics,Seed Bomb,Acrobatics,Low Kick,Acrobatics,Acrobatics,Low Kick,Acrobatics,Acrobatics,Acrobatics,Acrobatics,Acrobatics,Acrobatics,Low Kick,Acrobatics,Acrobatics,Acrobatics,Low Kick,Acrobatics,Acrobatics,Acrobatics
Ampharos,Dynamic Punch,NaN,NaN,Counter,Signal Beam,Power Gem,Signal Beam,Meteor Beam,Brick Break,Fire Punch,Breaking Swipe,Breaking Swipe,Dynamic Punch,Ice Punch,Ice Punch,Fire Punch,Fire Punch,Ice Punch,Dazzling Gleam,Signal Beam,Fire Punch,Iron Tail,Dragon Pulse,Dazzling Gleam,Dragon Pulse,...,Focus Blast,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Counter,Dazzling Gleam,Dragon Pulse,Dazzling Gleam,Dazzling Gleam,Dragon Pulse,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dragon Pulse,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam,Dragon Pulse,Dazzling Gleam,Dazzling Gleam,Dazzling Gleam
Araquanid,NaN,NaN,NaN,NaN,Mirror Coat,Poison Jab,Mirror Coat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Crunch,Mirror Coat,NaN,NaN,Giga Drain,Crunch,Giga Drain,...,Ice Beam,Crunch,Crunch,Crunch,Crunch,NaN,Crunch,Giga Drain,Crunch,Crunch,Giga Drain,Crunch,Crunch,Crunch,Crunch,Crunch,Crunch,Giga Drain,Crunch,Crunch,Crunch,Giga Drain,Crunch,Crunch,Crunch
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wobuffet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Counter,NaN,NaN,NaN,Mirror Coat,Counter,Mirror Coat,...,NaN,Counter,Counter,Counter,Counter,NaN,Counter,Mirror Coat,Counter,Counter,Mirror Coat,Counter,Counter,Counter,Counter,Counter,Counter,Mirror Coat,Counter,Counter,Counter,Mirror Coat,Counter,Counter,Counter
Wobuffet,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Counter,NaN,NaN,NaN,Mirror Coat,Counter,Mirror Coat,...,NaN,Counter,Counter,Counter,Counter,NaN,Counter,Mirror Coat,Counter,Counter,Mirror Coat,Counter,Counter,Counter,Counter,Counter,Counter,Mirror Coat,Counter,Counter,Counter,Mirror Coat,Counter,Counter,Counter
Wormadam-Trash,NaN,NaN,NaN,NaN,Sucker Punch,Shadow Ball,Sucker Punch

In [ ]:
# Again useless code I probably wrote as a part of my attempt to link types back to moves.
# On the plus side, this can probably be used to list out all moves of a certain type (after dropping duplicates of course).

def from_type_to_moves_deepseek(type):
    indices = np.where(type_df == type)
    row_indices, col_indices = indices
    col_indices_moves = col_indices - 3
    # Use advanced indexing on the numpy array to get the values directly
    return movesdf.to_numpy()[row_indices, col_indices_moves]

from_type_to_moves_deepseek('Typeless')

array(['Counter', 'Counter', 'Counter', 'Counter', 'Mirror Coat',
       'Mirror Coat', 'Mirror Coat', 'Mirror Coat', 'Mirror Coat',
       'Counter', 'Metal Burst', 'Counter', 'Mirror Coat', 'Counter',
       'Counter', 'Counter', 'Metal Burst', 'Mirror Coat', 'Mirror Coat',
       'Mirror Coat', 'Counter', 'Mirror Coat', 'Counter', 'Counter',
       'Mirror Coat', 'Counter', 'Counter', 'Counter', 'Counter',
       'Mirror Coat', 'Metal Burst', 'Metal Burst', 'Counter', 'Counter',
       'Mirror Coat', 'Counter', 'Counter', 'Counter', 'Mirror Coat',
       'Counter', 'Counter', 'Mirror Coat', 'Counter', 'Mirror Coat',
       'Mirror Coat', 'Counter', 'Counter', 'Counter', 'Counter',
       'Mirror Coat', 'Counter', 'Counter', 'Counter', 'Mirror Coat',
       'Counter', 'Counter', 'Counter', 'Counter', 'Counter',
       'Comeuppance', 'Counter', 'Comeuppance', 'Counter', 'Counter',
       'Counter', 'Counter', 'Counter', 'Counter', 'Counter', 'Counter',
       'Counter', 'Counter', 'Co

In [ ]:
#try **kwargs, maybe we can input keys as movenames and values as types later
def coverage_function(typelist):
    for i in typelist:
       print(offdf.loc[i])

In [ ]:
#trying the coverage function on a small segment, will do it to the entire database eventually

#type_df.apply(len, axis='columns') #the default is rows btw, which means it applies the function to each row, which is good


#type_df.apply(coverage_function, axis='columns') #We need to decide whether the axis should be columns or rows
'''abomasnow_df=type_df.loc['Abomasnow'].map(Off)
abomasnow_df'''

"abomasnow_df=type_df.loc['Abomasnow'].map(Off)\nabomasnow_df"

In [ ]:
'''coverage_df=type_df['MonType1'].map(Off)
coverage_df'''

"coverage_df=type_df['MonType1'].map(Off)\ncoverage_df"

In [ ]:
print(Off)

{'Typeless': [], 'Normal': [], 'Fire': ['Grass', 'Ice', 'Bug', 'Steel'], 'Water': ['Fire', 'Ground', 'Rock'], 'Grass': ['Water', 'Ground', 'Rock'], 'Electric': ['Water', 'Flying'], 'Ice': ['Grass', 'Ground', 'Flying', 'Dragon'], 'Fighting': ['Normal', 'Ice', 'Rock', 'Dark', 'Steel'], 'Poison': ['Grass', 'Fairy'], 'Ground': ['Fire', 'Electric', 'Poison', 'Rock', 'Steel', 'Dragon'], 'Flying': ['Grass', 'Fighting', 'Bug'], 'Psychic': ['Fighting', 'Poison'], 'Bug': ['Grass', 'Psychic', 'Dark'], 'Rock': ['Fire', 'Ice', 'Flying', 'Bug', 'Dragon'], 'Ghost': ['Psychic', 'Ghost'], 'Dragon': ['Dragon'], 'Dark': ['Psychic', 'Ghost'], 'Steel': ['Ice', 'Rock', 'Fairy'], 'Fairy': ['Fighting', 'Dragon', 'Dark'], 'nan': []}


In [ ]:
#full_df_attempt=type_df.replace(Off)
#full_df_attempt

In [ ]:
#apparently the above code, using replace instead of applymap is better(more efficient), but for that you need to remove null values. 
#I want to push myself to see how long I can deal with null values
full_df_attempt=type_df.map(lambda x: Off.get(x))
full_df_attempt
#LESSSSGOOOOOO

,MonType1,MonType2,MonType3,Type1,Type2,Type3,Type4,Type5,Type6,Type7,Type8,Type9,Type10,Type11,Type12,Type13,Type14,Type15,Type16,Type17,Type18,Type19,Type20,Type21,Type22,Type23,Type24,Type25,Type26,Type27,Type28,Type29,Type30,Type31,Type32,Type33,Type34,Type35,Type36,Type37,Type38,Type39,Type40,Type41,Type42,Type43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,"[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]",None,"[Normal, Ice, Rock, Dark, Steel]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Ice, Rock, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Absol,"[Psychic, Ghost]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Ice, Rock, Fairy]","[Grass, Psychic, Dark]","[Fighting, Dragon, Dark]","[Fighting, Poison]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Fire, Ice, Flying, Bug, Dragon]","[Grass, Psychic, Dark]","[Fighting, Poison]","[Grass, Fighting, Bug]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]","[Water, Flying]","[Fighting, Poison]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Accelgor,"[Grass, Psychic, Dark]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Fire, Ground, Rock]","[Psychic, Ghost]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Fairy]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aerodactyl,"[Fire, Ice, Flying, Bug, Dragon]","[Grass, Fighting, Bug]",None,"[Fire, Ground, Rock]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Ice, Rock, Fairy]","[Fighting, Poison]","[Water, Flying]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aggron,"[Ice, Rock, Fairy]","[Fire, Ice, Flying, Bug, Dragon]",None,"[Fire, Ground, Rock]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Water, Flying]","[Psychic, Ghost]",[Dragon],"[Psychic, Ghost]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Water, Flying]","[Fire, Ground, Rock]","[Fire, Electric, Poison, Rock, Steel, Dragon]",[Dragon],"[Normal, Ice, Rock, Dark, Steel]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[],"[Fire, Electric, Poison, Rock, Steel, Dragon]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,"[Grass, Psychic, Dark]","[Grass, Fighting, Bug]",None,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,No

In [ ]:
full_df_attempt.loc['Ambipom']

MonType1                                  []
MonType2                                None
MonType3                                None
Type1                 [Grass, Fighting, Bug]
Type2       [Normal, Ice, Rock, Dark, Steel]
                          ...               
Type39                                  None
Type40                                  None
Type41                                  None
Type42                                  None
Type43                                  None
Name: Ambipom, Length: 46, dtype: object

In [ ]:
type(full_df_attempt.loc['Ambipom'])

pandas.core.series.Series

In [ ]:
type(type_df.iloc[400,10])

str

In [ ]:
type(full_df_attempt.iloc[400,10])

list

In [ ]:
(type_df.drop(columns=['MonType1','MonType2','MonType3'])).columns

Index(['Type1', 'Type2', 'Type3', 'Type4', 'Type5', 'Type6', 'Type7', 'Type8',
       'Type9', 'Type10', 'Type11', 'Type12', 'Type13', 'Type14', 'Type15',
       'Type16', 'Type17', 'Type18', 'Type19', 'Type20', 'Type21', 'Type22',
       'Type23', 'Type24', 'Type25', 'Type26', 'Type27', 'Type28', 'Type29',
       'Type30', 'Type31', 'Type32', 'Type33', 'Type34', 'Type35', 'Type36',
       'Type37', 'Type38', 'Type39', 'Type40', 'Type41', 'Type42', 'Type43'],
      dtype='object')

In [ ]:
nonstab_columns=(type_df.drop(columns=['MonType1','MonType2','MonType3'])).columns
typeslist=pd.concat([type_df.loc[:, col] for col in nonstab_columns], axis = 0, ignore_index=True)
typeslist

0        Fighting
1        Fighting
2        Fighting
3           Water
4           Water
           ...   
20979         NaN
20980         NaN
20981         NaN
20982         NaN
20983         NaN
Length: 20984, dtype: object

In [ ]:
typeslist=typeslist.dropna()
len(typeslist) #this is 17982 without dropping empty cells, and 1127 after dropping

6727

In [ ]:
pd.set_option('display.max_rows', None)
smthh=moveslist.drop_duplicates()
smthh.columns=['Move']
smthh=smthh.sort_values()
smthh=(smthh.reset_index()
        .drop(columns='index'))
smthh
#make sure to compare the number of things in this list and the below list to make sure no extra faulty moves got added

,0
0,Acrobatics
1,Aerial Ace
2,Air Slash
3,Alluring Voice
4,Aqua Cutter
5,Aqua Jet
6,Aqua Tail
7,Assurance
8,Aura Sphere
9,Aurora Beam


In [ ]:
#Temporary code, apparently one of the moves is named 'Dark', which means I messed up somwehere, so I'm trying to find the index of the place I messed up so that I can edit it in Excel
#once I fix it, this code will be useless.
indices=np.where(movesdf=='Uturn')

row_indices, col_indices = indices[0], indices[1]
print(row_indices,col_indices)

#so apparently on row 32, columns 11 and 12, there's a move named Dark

'''
movesdf.iloc[32,11]
movesdf.iloc[32,12]

print(df.loc[32,'Move12'])
print(df.loc[32,'Move13'])

print(df.loc[32,'Name'])
'''
#print(df.loc[397,'Name'])


for i,j in zip(row_indices,col_indices):
    if pd.isnull(type_df.iloc[i,j]):
        print(df.loc[i,'Name'],movesdf.iloc[i,j])


#Okay it's Barraskewda, time to fix it

[  7  16  23  24  29  49  52  53  66  71  75  76  82  91  95  97  98  99
 100 108 127 134 144 146 148 149 150 154 158 159 179 185 189 194 201 204
 205 206 210 215 222 232 236 239 252 259 263 267 273 283 293 296 303 304
 305 306 309 313 315 316 317 318 319 320 330 342 343 384 402 404 417 418
 420 429 431 434 437 439 443 444 451 452 471 474 482 485 487] [11  9  3  1 15  4  4  8  7  3  8  8  4  9 12 11  8  6  6 19 25  4 10  4
  9  4  4 22  8 23  7 14 13  7 14 17  7 11  4  0 16  8 23  6  3  7 20 14
  9  7 15 11  3  3  3  3  8 12  5 17 15  9  8  2 10 13 13 19  4  3  4  7
  2  6 16  6 17  9  7  7  2 17  4  3 10  7 13]
Pidgeot Uturn
Talonflame Uturn
Unfezant Uturn


In [ ]:
#if not isinstance(moveslist,pd.DataFrame):
    #moveslist2=moveslist.to_frame(name='Move')
moveslist2=moveslist.to_frame(name='Move')
moveslist2['Type']=typeslist
#moveslist2=moveslist2[:-1] #there's something extremely weird with the last row
moveslist2=(moveslist2.drop_duplicates()
            .sort_values(by='Move')
            .reset_index()
            .drop(columns='index')
            .set_index('Move'))
moveslist2


,Type
Move,
Acrobatics,Flying
Aerial Ace,Flying
Air Slash,Flying
Alluring Voice,Fairy
Aqua Cutter,Water
Aqua Jet,Water
Aqua Tail,Water
Assurance,Dark
Aura Sphere,Fighting


#uturn correspond to NaN,

 fighting type counter



In [ ]:
move_dict=moveslist2['Type'].to_dict()   #if I directly convert moveslist2 to dictionary, it makes a nested dictionary with the type column
move_dict

{'Acrobatics': 'Flying',
 'Aerial Ace': 'Flying',
 'Air Slash': 'Flying',
 'Alluring Voice': 'Fairy',
 'Aqua Cutter': 'Water',
 'Aqua Jet': 'Water',
 'Aqua Tail': 'Water',
 'Assurance': 'Dark',
 'Aura Sphere': 'Fighting',
 'Aurora Beam': 'Ice',
 'Avalanche': 'Ice',
 'Axe Kick': 'Fighting',
 'Beat Up': 'Dark',
 'Belch': 'Poison',
 'Bite': 'Dark',
 'Blaze Kick': 'Fire',
 'Body Press': 'Fighting',
 'Bone Club': 'Ground',
 'Bone Rush': 'Ground',
 'Bonemerang': 'Ground',
 'Boomburst': 'Normal',
 'Bounce': 'Flying',
 'Brave Bird': 'Flying',
 'Breaking Swipe': 'Dragon',
 'Brick Break': 'Fighting',
 'Brine': 'Water',
 'Brutal Swing': 'Dark',
 'Bubble Beam': 'Water',
 'Bug Buzz': 'Bug',
 'Bulldoze': 'Earthquake',
 'Bullet Punch': 'Steel',
 'Bullet Seed': 'Grass',
 'Burning Jealousy': 'Fire',
 'Chilling Water': 'Water',
 'Close Combat': 'Fighting',
 'Comeuppance': 'Typeless',
 'Counter': 'Typeless',
 'Crabhammer': 'Water',
 'Cross Chop': 'Fighting',
 'Cross Poison': 'Poison',
 'Crunch': 'Dark',


In [ ]:
pd.set_option('display.max_rows', 20)


maybe make a list of slow mons with priority moves later? or just mons with priority moves. So that extreme speed etc can be included

map the moves to the mons, so that maybe later we can just randomly filter the mons with priority moves who generally have coverage against the elite 4

oh wait, maybe we can make a column for mons called, "Priority", and inside that we can put in a list of the priority moves they have, others will be null

so then in our regular filter we can just see good e4 mons, and also particularly see the good e4 mons which have priority

find a way to sort mons such that mons with broaaaaad matching coverage have lower priority, and mons with specific coverage towards the particular e4 are higher priority. 
Cuz like florges has only bug, psychic, grass, and fairy moves, so the moment you get an elite aligned towards these types, we badly want to use florges cuz it won't get much opportunity otherwise


okay I'm deciding on the physical special split by using base stats. Also sorting the mons based on base stats.
I want to give some higher weightage to huge power, iron fist, and sheer force.


In [ ]:

la=pd.Index(['MonType1','MonType2','MonType3']) #adding stab types to the front of the list
la=la.append(df.columns[~(df.columns.str.contains('Type', na=False))]) #the rest of the list has everything but the types
#la is a random temp var
clean_df=df[la]
clean_df

,MonType1,MonType2,MonType3,Name,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,...,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
0,Grass,Ice,NaN,Abomasnow,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,Fling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dark,NaN,NaN,Absol,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Bug,NaN,NaN,Accelgor,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,Pursuit,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Rock,Flying,NaN,Aerodactyl,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,Pursuit,Bulldoze,Payback,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Steel,Rock,NaN,Aggron,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,...,Focus Blast,Ice Beam,Thunderbolt,Surf,Bulldoze,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,Dig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483,Bug,Flying,NaN,Yanmega,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,Pursuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484,Normal,NaN,NaN,Zangoose,RU,NaN,NaN,NaN,Counter,Reversal,Fling,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,...,Throat Chop,X Scissor,Zen Headbutt,Dig,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,Rock Tomb,Payback,Pursuit,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
485,Dark,Grass,NaN,Zarude,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,Dig,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,Rock Tomb,Acrobatics,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
486,Electric,NaN,NaN,Zebstrika,RU,NaN,NaN,NaN,Overheat,Low Kick,High Horsepower,Bounce,Signal Beam,Smart Strike,Bulldoze,Pursuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_df=clean_df.set_index('Name')

In [ ]:
clean_df

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,Fling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,Pursuit,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,Pursuit,Bulldoze,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,Bulldoze,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,Dig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,Pursuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,Fling,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,Dig,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,Rock Tomb,Payback,Pursuit,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,Dig,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,Rock Tomb,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_df.loc[:,'Move1':].map(lambda x: Off.get(move_dict.get(x))) #every value in the serieses chosen are keys in the dictionaries, which is why we can be comfortable
#we can do clean_df.iloc[:,7:] = clean_df.iloc[:,7:].map later
#the 'Move1' is so that the first 6 columns stay untouched and don't get their data deleted

#let's not modify clean_df, let's create a copy called processed df or something so that we can always link back to clean df through an index or something

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,"[Normal, Ice, Rock, Dark, Steel]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Ice, Rock, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Absol,"[Normal, Ice, Rock, Dark, Steel]","[Ice, Rock, Fairy]","[Grass, Psychic, Dark]","[Fighting, Dragon, Dark]","[Fighting, Poison]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Fire, Ice, Flying, Bug, Dragon]","[Grass, Psychic, Dark]","[Fighting, Poison]","[Grass, Fighting, Bug]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]","[Water, Flying]","[Fighting, Poison]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Accelgor,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Fire, Ground, Rock]","[Psychic, Ghost]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Fairy]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aerodactyl,"[Fire, Ground, Rock]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Ice, Rock, Fairy]","[Fighting, Poison]","[Water, Flying]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]",None,"[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aggron,"[Fire, Ground, Rock]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Water, Flying]","[Psychic, Ghost]",[Dragon],"[Psychic, Ghost]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Water, Flying]","[Fire, Ground, Rock]",None,[Dragon],"[Normal, Ice, Rock, Dark, Steel]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[],"[Fire, Electric, Poison, Rock, Steel, Dragon]",None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Zangoose,[],"[Normal, Ice, Rock, Dark, Steel]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ground, Rock]","[Water, Flying]","[Normal, Ice, Rock, Da

In [ ]:
clean_df.loc[:,['MonType1','MonType2','MonType3']].map(lambda x: Off.get(x))
#these are the stab types which only need to be passed through one dictionary, not a nested dictionary

,MonType1,MonType2,MonType3
Name,,,
Abomasnow,"[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]",None
Absol,"[Psychic, Ghost]",None,None
Accelgor,"[Grass, Psychic, Dark]",None,None
Aerodactyl,"[Fire, Ice, Flying, Bug, Dragon]","[Grass, Fighting, Bug]",None
Aggron,"[Ice, Rock, Fairy]","[Fire, Ice, Flying, Bug, Dragon]",None
...,...,...,...
Yanmega,"[Grass, Psychic, Dark]","[Grass, Fighting, Bug]",None
Zangoose,[],None,None
Zarude,"[Psychic, Ghost]","[Water, Ground, Rock]",None


In [ ]:
clean_df

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,Fling,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,Pursuit,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,Pursuit,Bulldoze,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,Bulldoze,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,Dig,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,Pursuit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,Fling,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,Dig,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,Rock Tomb,Payback,Pursuit,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,Dig,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,Rock Tomb,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#temporary code, a sort of filter, to remove fling and all
clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=np.NaN) #filling all of these moves with NaN, deleting them
clean_df

C:\Users\Srijan Badhya\AppData\Local\Temp\ipykernel_22032\3459290613.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  clean_df=clean_df.replace(to_replace=["Fling","Pursuit","Bulldoze","Rock Tomb","Dig"],value=np.NaN) #filling all of these moves with NaN, deleting them


,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,NaN,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,NaN,Payback,NaN,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,NaN,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,NaN,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

mid=clean_df.apply(lambda x: x=='Brick Break',axis=1)

mid[mid.any(axis='columns')] #the any function filters the places where at least one value is true
#these are all the mons with brick break

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ambipom,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ampharos,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Armaldo,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Azumarill,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volbeat,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Volcanion,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Wigglytuff,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
mid=clean_df.apply(lambda x: x.isin(['Brick Break','Aqua Tail']),axis=1)

mid[mid.any(axis='columns')] #the any function filters the places where at least one value is true
#these are all the mons with brick break

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aerodactyl,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aggron,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ambipom,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ampharos,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volcanion,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Watchog,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Wigglytuff,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:

def brickbreakthing(x):
    return x.isin(['Brick Break'])

mid=clean_df.apply(brickbreakthing,axis=1)

mid[mid.any(axis='columns')] #the any function filters the places where at least one value is true
#these are all the mons with brick break

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ambipom,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ampharos,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Armaldo,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Azumarill,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volbeat,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Volcanion,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Wigglytuff,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#mid=clean_df
mid =clean_df.loc[:,'Move1':].map(lambda x: Off.get(move_dict.get(x)))
mid

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,"[Normal, Ice, Rock, Dark, Steel]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Ice, Rock, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Absol,"[Normal, Ice, Rock, Dark, Steel]","[Ice, Rock, Fairy]","[Grass, Psychic, Dark]","[Fighting, Dragon, Dark]","[Fighting, Poison]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Fire, Ice, Flying, Bug, Dragon]","[Grass, Psychic, Dark]","[Fighting, Poison]","[Grass, Fighting, Bug]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]","[Water, Flying]","[Fighting, Poison]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Accelgor,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Fire, Ground, Rock]","[Psychic, Ghost]",None,"[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Fairy]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aerodactyl,"[Fire, Ground, Rock]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Ice, Rock, Fairy]","[Fighting, Poison]","[Water, Flying]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]",None,None,"[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aggron,"[Fire, Ground, Rock]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Water, Flying]","[Psychic, Ghost]",[Dragon],"[Psychic, Ghost]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Water, Flying]","[Fire, Ground, Rock]",None,[Dragon],"[Normal, Ice, Rock, Dark, Steel]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Zangoose,[],"[Normal, Ice, Rock, Dark, Steel]",None,"[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ground, Rock]","[Water, Flying]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Fairy]","[Water, Ground, Rock]","[Psychic, Ghost]","[Psychic, Ghost]","[Grass, Psychic, Dark

In [ ]:
#mid =clean_df.loc[:,'Move1':].map(lambda x: Off.get(move_dict.get(x)))
mid=clean_df

def typechecker(x,axis=1):
    #if (pd.isna(x)):
    #    return False
    
    return x.isin(['Brick Break'])
    


mid=mid.apply(typechecker, axis=1) #it goes down every column then onto the next column

mid[mid.any(axis='columns')] #the any function filters the places where at least one value is true
#these are all the mons with brick brek OR aqua tail

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ambipom,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Ampharos,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Armaldo,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Azumarill,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Volbeat,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Volcanion,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Wigglytuff,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
mid =clean_df.loc[:,'Move1':].map(lambda x: Off.get(move_dict.get(x)))

In [ ]:
mask = mid.loc['Abomasnow'].apply(lambda cell: 'Rock' in cell if isinstance(cell, list) else False)
mask


Move1        True
Move2        True
Move3        True
Move4        True
Move5        True
            ...  
Move38.1    False
Move40      False
Move41      False
Move42      False
Move43      False
Name: Abomasnow, Length: 43, dtype: bool

In [ ]:
boolean_mask = mid.map(lambda cell: 'Rock' in cell if isinstance(cell, list) else False)
boolean_mask

,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Absol,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Accelgor,True,True,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aerodactyl,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aggron,True,True,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Zangoose,False,True,False,True,True,True,False,True,True,False,False,True,True,False,True,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
Zarude,True,True,True,True,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
clean_df

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,NaN,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,NaN,Payback,NaN,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,NaN,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,NaN,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#enough messing around, let's make a proper dataframe

coverage=clean_df.drop(columns=['Tier','In Wishlist','Used','Trash'])
coverage.loc[:,'Move1':]=coverage.loc[:,'Move1':].map(lambda x: move_dict.get(x)) #converts the move names to their types


C:\Users\Srijan Badhya\AppData\Local\Temp\ipykernel_22032\627572130.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None None None None None None None None None None
 None None None None None N

In [ ]:
coverage=coverage.map(lambda x: Off.get(x))
coverage

,MonType1,MonType2,MonType3,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,"[Water, Ground, Rock]","[Grass, Ground, Flying, Dragon]",None,"[Normal, Ice, Rock, Dark, Steel]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Ice, Rock, Fairy]","[Normal, Ice, Rock, Dark, Steel]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Absol,"[Psychic, Ghost]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Ice, Rock, Fairy]","[Grass, Psychic, Dark]","[Fighting, Dragon, Dark]","[Fighting, Poison]","[Fire, Ice, Flying, Bug, Dragon]","[Psychic, Ghost]","[Fire, Ice, Flying, Bug, Dragon]","[Grass, Psychic, Dark]","[Fighting, Poison]","[Grass, Fighting, Bug]","[Grass, Ice, Bug, Steel]","[Psychic, Ghost]","[Water, Flying]","[Fighting, Poison]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Accelgor,"[Grass, Psychic, Dark]",None,None,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Grass, Fairy]","[Fire, Ground, Rock]","[Psychic, Ghost]",None,"[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Fairy]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aerodactyl,"[Fire, Ice, Flying, Bug, Dragon]","[Grass, Fighting, Bug]",None,"[Fire, Ground, Rock]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Ice, Rock, Fairy]","[Fighting, Poison]","[Water, Flying]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]",None,None,"[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Aggron,"[Ice, Rock, Fairy]","[Fire, Ice, Flying, Bug, Dragon]",None,"[Fire, Ground, Rock]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Psychic, Ghost]",[Dragon],"[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Grass, Ground, Flying, Dragon]","[Normal, Ice, Rock, Dark, Steel]",[Dragon],"[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]","[Water, Flying]","[Psychic, Ghost]",[Dragon],"[Psychic, Ghost]","[Fire, Electric, Poison, Rock, Steel, Dragon]","[Grass, Ice, Bug, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Grass, Ground, Flying, Dragon]","[Water, Flying]","[Fire, Ground, Rock]",None,[Dragon],"[Normal, Ice, Rock, Dark, Steel]","[Psychic, Ghost]","[Normal, Ice, Rock, Dark, Steel]",[],None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,"[Grass, Psychic, Dark]","[Grass, Fighting, Bug]",None,"[Normal, Ice, Rock, Dark, Steel]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]","[Psychic, Ghost]","[Water, Ground, Rock]","[Fighting, Poison]","[Psychic, Ghost]",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
Zangoose,[],None,None,[],"[Normal, Ice, Rock, Dark, Steel]",None,"[Normal, Ice, Rock, Dark, Steel]","[Normal, Ice, Rock, Dark, Steel]","[Water, G

In [ ]:
#boolean_mask = coverage.map(lambda cell: 'Rock' in cell or 'Flying' in cell if isinstance(cell, list) else False)
#it can't be an OR operation, it has to be an AND operation sequentially
#maybe filter based on which rows have 'any' true, and then further filter with the next type

#remember, we don't want mons which are good against at least one type, we want mons which can be found in the filters of all types individually
boolean_mask = coverage.map(lambda cell: 'Rock' in cell if isinstance(cell, list) else False)
boolean_mask

,MonType1,MonType2,MonType3,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,True,False,False,True,True,True,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Absol,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Accelgor,False,False,False,True,True,False,True,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aerodactyl,False,False,False,True,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aggron,True,False,False,True,True,False,False,False,True,False,False,True,False,False,True,False,False,False,False,True,False,True,False,False,True,False,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Zangoose,False,False,False,False,True,False,True,True,True,False,True,True,False,False,True,True,False,True,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False
Zarude,False,True,False,True,True,True,True,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
clean_df[boolean_mask]

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,NaN,NaN,NaN,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,NaN,NaN,Body Press,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Close Combat,Iron Tail,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Focus Blast,Energy Ball,NaN,Water Shuriken,NaN,NaN,Drain Punch,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aqua Tail,NaN,NaN,Earthquake,NaN,NaN,Iron Head,NaN,NaN,NaN,Earth Power,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,NaN,NaN,NaN,NaN,NaN,NaN,Aqua Tail,Body Press,NaN,NaN,NaN,Earthquake,NaN,NaN,Low Kick,NaN,NaN,Superpower,NaN,NaN,NaN,NaN,Earth Power,NaN,Focus Blast,NaN,NaN,Surf,NaN,NaN,Dynamic Punch,NaN,Reversal,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reversal,Solar Beam,NaN,NaN,NaN,Giga Drain,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,NaN,Low Kick,Solar Beam,NaN,NaN,Dynamic Punch,Surf,NaN,Aura Sphere,NaN,Seed Bomb,NaN,NaN,NaN,NaN,NaN,Brick Break,NaN,Giga Drain,NaN,NaN,NaN,NaN,NaN,NaN,Low Sweep,Revenge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,NaN,Grass,NaN,NaN,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,NaN,NaN,Stomping Tantrum,NaN,Low Sweep,Revenge,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_df[boolean_mask.any(axis='columns')] #.any gives one output value per row, and using that as input to clean_df selects the relevant rows
#but prints out all the contents of these rows
#so better to create a new column called 'any', or rather, the 'type', so that we can use it later if we wish

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,NaN,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,NaN,Payback,NaN,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,NaN,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,NaN,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:

elite_1='Ground'
elite_2='Psychic'
elite_3='Fairy'
elite_4='Fire'



mask_1 = coverage.map(lambda cell: elite_1 in cell if isinstance(cell, list) else False)
mask_2 = coverage.map(lambda cell: elite_2 in cell if isinstance(cell, list) else False)
mask_3 = coverage.map(lambda cell: elite_3 in cell if isinstance(cell, list) else False)
mask_4 = coverage.map(lambda cell: elite_4 in cell if isinstance(cell, list) else False)

In [ ]:
clean_df[mask_1.any(axis='columns')]
#256 rows here

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Alcremie,Fairy,NaN,NaN,RU,NaN,NaN,NaN,Energy Ball,Giga Drain,Mystical Fire,Psychic,Psyshock,Stored Power,Drain Punch,NaN,Solar Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Xatu,Psychic,Flying,NaN,RU,NaN,NaN,NaN,Grass Knot,Solar Beam,Foul Play,Heat Wave,Dazzling Gleam,Shadow Ball,Giga Drain,Signal Beam,Steel Wing,Sucker Punch,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,NaN,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,NaN,Payback,NaN,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
clean_df[mask_2.any(axis='columns')]
#241 rows here

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,Dark,NaN,NaN,RU,NaN,NaN,NaN,Close Combat,Iron Tail,Megahorn,Play Rough,Psycho Cut,Rock Slide,Shadow Claw,Stone Edge,X Scissor,Zen Headbutt,Bounce,Flamethrower,Shadow Ball,Thunderbolt,Future Sight,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,Bug,Flying,NaN,RU,NaN,NaN,NaN,Reversal,Solar Beam,Psychic,Crunch,Shadow Ball,Giga Drain,Psychic Noise,Night Slash,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,Normal,NaN,NaN,RU,NaN,NaN,NaN,Counter,Reversal,NaN,Close Combat,Focus Blast,Grass Knot,Gunk Shot,Low Kick,Solar Beam,Ice Beam,Flamethrower,Dynamic Punch,Surf,Thunderbolt,Aura Sphere,Poison Jab,Seed Bomb,Shadow Ball,Throat Chop,X Scissor,Zen Headbutt,NaN,Brick Break,Fire Punch,Giga Drain,Ice Punch,Rock Slide,Thunder Punch,Shadow Claw,Night Slash,Knock Off,Low Sweep,Revenge,NaN,Payback,NaN,Power Trip,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,Dark,Grass,NaN,RU,NaN,NaN,NaN,Close Combat,Low Kick,Hammer Arm,Iron Tail,NaN,Rock Slide,Stomping Tantrum,Uturn,Low Sweep,Revenge,NaN,Acrobatics,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mask_1[mask_2]
#mask 1 and mask 2 are ice and fire, and here we can see that aerodactyl's first type which is rock, is true
#this is incorrect code, we're not trying to find the moves which are supereffective against all 4 types, we want the mons which are good against all
#so we'll use the any() function, get a list of mons, and then multiply and nest like this

,MonType1,MonType2,MonType3,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Absol,False,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,False,NaN,NaN,NaN,NaN,NaN,False,False,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Zangoose,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,False,False,NaN,NaN,NaN,False,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN
Zarude,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
mask=(mask_2.loc[mask_1.any(axis='columns')].any(axis='columns'))
mask.value_counts()
#there are 185 Trues here
mask

Name
Abomasnow      True
Accelgor       True
Aerodactyl     True
Aggron         True
Alcremie      False
              ...  
Xatu           True
Yanmega        True
Zangoose       True
Zarude         True
Zoroark        True
Length: 407, dtype: bool

In [ ]:
clean_df[(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))]
#we don't need to nest, we can just have an AND condition
#we've got the desired list of mons with moves against all types

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,Brick Break,Earthquake,Earth Power,Iron Tail,Low Kick,Rock Slide,Shadow Ball,Body Press,Outrage,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,Focus Blast,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,Drain Punch,Reversal,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,Flying,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,Dragon Claw,Earthquake,Fire Fang,Ice Fang,Iron Head,Psychic Fangs,Thunder Fang,Dragon Pulse,Earth Power,Flamethrower,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,Body Press,Avalanche,Crunch,Dragon Claw,Earthquake,Fire Punch,Ice Punch,Low Kick,Outrage,Shadow Claw,Superpower,Thunder Punch,Payback,Dragon Pulse,Dark Pulse,Earth Power,Flamethrower,Focus Blast,Ice Beam,Thunderbolt,Surf,NaN,Dragon Rush,Dynamic Punch,Payback,Reversal,Counter,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ambipom,Normal,NaN,NaN,RU,NaN,NaN,NaN,Acrobatics,Low Kick,Fire Punch,Foul Play,Gunk Shot,Ice Punch,Knock Off,Seed Bomb,Shadow Claw,Thunder Punch,Triple Axel,Uturn,Dual Chop,Iron Tail,Fury Cutter,Trailblaze,Smack Down,Payback,Brick Break,Chilling Water,Counter,Grass Knot,NaN,Shadow Ball,Throat Chop,Thunderbolt,Dynamic Punch,NaN,Low Sweep,Payback,Revenge,Shock Wave,Smack Down,Zap Cannon,Bounce,NaN,Beat Up,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wigglytuff,Normal,Fairy,NaN,RU,NaN,NaN,NaN,Counter,Gyro Ball,Grass Knot,Solar Beam,Zap Cannon,Dynamic Punch,Energy Ball,Flamethrower,Ice Beam,Psychic,Thunderbolt,Wild Charge,Bounce,Body Press,Dark Pulse,Expanding Force,Ice Spinner,Psyshock,Shadow Ball,Zen Headbutt,NaN,Brick Break,Drain Punch,Fire Punch,Psychic Noise,Thunder Punch,Ice Punch,Wake Up Slap,Knock Off,Bubble Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wishiwashi,Water,NaN,NaN,RU,NaN,NaN,NaN,Earthquake,Iron Tail,Ice Beam,Uturn,Scale Shot,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wyrdeer,Normal,Psychic,NaN,RU,NaN,Yes,NaN,Megahorn,Solar Beam,Earthquake,Iron Tail,Jump Kick,High Horsepower,Earth Power,Energy Ball,Thunderbolt,Wild Charge,Bounce,Lunge,Shadow Ball,Throat Chop,NaN,Signal Beam,Sucker Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1))
#now we need to remove the random ass moves, and get the list of all the moves that will be uased against the elite
#this will be an OR operation of the masks
chosen_mons

Name
Abomasnow      True
Absol         False
Accelgor       True
Aerodactyl     True
Aggron         True
              ...  
Yanmega       False
Zangoose       True
Zarude         True
Zebstrika     False
Zoroark       False
Length: 488, dtype: bool

In [ ]:
mask_1 | mask_2
#move 1 of abomasnow is good against ice, and move 2 is good against fire, so this is correct

,MonType1,MonType2,MonType3,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,True,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Absol,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Accelgor,True,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aerodactyl,False,False,False,True,True,False,False,False,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Aggron,False,False,False,True,False,True,True,False,False,False,True,False,False,True,False,False,True,False,True,False,False,False,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Yanmega,True,False,False,False,True,False,True,True,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
Zangoose,False,False,False,False,False,False,False,False,True,False,False,True,True,False,False,True,False,False,False,True,True,True,True,False,False,False,False,True,True,False,False,True,True,True,False,False,False,True,False,True,False,False,False,False,False,False
Zarude,True,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
#clean_df[(mask_1 | mask_2 | mask_3 | mask_4)] #this prints all 437 rows, and prints mons good against at least one type
final =clean_df[(mask_1 | mask_2 | mask_3 | mask_4)].loc[chosen_mons] #this combines our old AND operation, with this current OR operation
#so this prints mons which are good against ALL types, but moves good against AT LEAST one type
final

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,NaN,NaN,NaN,NaN,NaN,Earthquake,Earth Power,Iron Tail,NaN,Rock Slide,Shadow Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,NaN,NaN,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN,NaN,NaN,NaN,NaN,Aqua Tail,Crunch,NaN,Earthquake,NaN,Ice Fang,Iron Head,NaN,NaN,NaN,Earth Power,NaN,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,NaN,NaN,NaN,NaN,Aqua Tail,NaN,Avalanche,Crunch,NaN,Earthquake,NaN,Ice Punch,NaN,NaN,Shadow Claw,NaN,NaN,Payback,NaN,Dark Pulse,Earth Power,NaN,NaN,Ice Beam,NaN,Surf,NaN,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ambipom,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Foul Play,Gunk Shot,Ice Punch,Knock Off,Seed Bomb,Shadow Claw,NaN,Triple Axel,Uturn,NaN,Iron Tail,Fury Cutter,Trailblaze,Smack Down,Payback,NaN,Chilling Water,NaN,Grass Knot,NaN,Shadow Ball,Throat Chop,NaN,NaN,NaN,NaN,Payback,NaN,NaN,Smack Down,NaN,NaN,NaN,Beat Up,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wigglytuff,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Gyro Ball,Grass Knot,Solar Beam,NaN,NaN,Energy Ball,NaN,Ice Beam,NaN,NaN,NaN,NaN,NaN,Dark Pulse,NaN,Ice Spinner,NaN,Shadow Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ice Punch,NaN,Knock Off,Bubble Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wishiwashi,Water,NaN,NaN,NaN,NaN,NaN,NaN,Earthquake,Iron Tail,Ice Beam,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wyrdeer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Megahorn,Solar Beam,Earthquake,Iron Tail,NaN,High Horsepower,Earth Power,Energy Ball,NaN,NaN,NaN,Lunge,Shadow Ball,Throat Chop,NaN,Signal Beam,Sucker Punch,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#let's combine all these columns into one column for the dataframe, also so that we don't have to dig through NaN stuff
#final['Useful_attributes']=final[final.columns].fillna('').agg(', '.join,axis=1)
#this code causes wayy too many extra commas from the null cells

#final['Useful_attributes']
#seems messed up, the next cell has a fix

In [ ]:
#the commas are extra, here's code by deepseek to avoid the commas
final['Useful_attributes'] = final[final.columns].apply(
    lambda row: ', '.join(str(item) for item in row if pd.notna(item)),
    axis=1
)
final['Useful_attributes']

Name
Abomasnow     Grass, Ice, Earthquake, Earth Power, Iron Tail...
Accelgor      Bug, Energy Ball, Sludge Bomb, Water Shuriken,...
Aerodactyl    Rock, Aqua Tail, Crunch, Earthquake, Ice Fang,...
Aggron        Steel, Rock, Aqua Tail, Avalanche, Crunch, Ear...
Ambipom       Foul Play, Gunk Shot, Ice Punch, Knock Off, Se...
                                    ...                        
Wigglytuff    Gyro Ball, Grass Knot, Solar Beam, Energy Ball...
Wishiwashi        Water, Earthquake, Iron Tail, Ice Beam, Uturn
Wyrdeer       Megahorn, Solar Beam, Earthquake, Iron Tail, H...
Zangoose      Grass Knot, Gunk Shot, Solar Beam, Ice Beam, S...
Zarude        Dark, Grass, Iron Tail, Rock Slide, Stomping T...
Name: Useful_attributes, Length: 235, dtype: object

In [ ]:
# I still need to add all those wishlist, used,tier values back
final[['Tier','In Wishlist','Used','Trash']]=clean_df[['Tier','In Wishlist','Used','Trash']]
final

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,Move18,...,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43,Useful_attributes
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,NaN,Earthquake,Earth Power,Iron Tail,NaN,Rock Slide,Shadow Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Grass, Ice, Earthquake, Earth Power, Iron Tail..."
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,NaN,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,NaN,NaN,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Bug, Energy Ball, Sludge Bomb, Water Shuriken,..."
Aerodactyl,Rock,NaN,NaN,RU,NaN,NaN,NaN,Aqua Tail,Crunch,NaN,Earthquake,NaN,Ice Fang,Iron Head,NaN,NaN,NaN,Earth Power,NaN,NaN,NaN,Payback,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Rock, Aqua Tail, Crunch, Earthquake, Ice Fang,..."
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,Aqua Tail,NaN,Avalanche,Crunch,NaN,Earthquake,NaN,Ice Punch,NaN,NaN,Shadow Claw,NaN,NaN,Payback,NaN,Dark Pulse,Earth Power,NaN,...,Ice Beam,NaN,Surf,NaN,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Steel, Rock, Aqua Tail, Avalanche, Crunch, Ear..."
Ambipom,NaN,NaN,NaN,RU,NaN,NaN,NaN,NaN,NaN,NaN,Foul Play,Gunk Shot,Ice Punch,Knock Off,Seed Bomb,Shadow Claw,NaN,Triple Axel,Uturn,NaN,Iron Tail,Fury Cutter,Trailblaze,Smack Down,Payback,...,Chilling Water,NaN,Grass Knot,NaN,Shadow Ball,Throat Chop,NaN,NaN,NaN,NaN,Payback,NaN,NaN,Smack Down,NaN,NaN,NaN,Beat Up,NaN,NaN,NaN,NaN,NaN,NaN,"Foul Play, Gunk Shot, Ice Punch, Knock Off, Se..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wigglytuff,NaN,NaN,NaN,RU,NaN,NaN,NaN,NaN,Gyro Ball,Grass Knot,Solar Beam,NaN,NaN,Energy Ball,NaN,Ice Beam,NaN,NaN,NaN,NaN,NaN,Dark Pulse,NaN,Ice Spinner,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ice Punch,NaN,Knock Off,Bubble Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Gyro Ball, Grass Knot, Solar Beam, Energy Ball..."
Wishiwashi,Water,NaN,NaN,RU,NaN,NaN,NaN,Earthquake,Iron Tail,Ice Beam,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Water, Earthquake, Iron Tail, Ice Beam, Uturn"
Wyrdeer,NaN,NaN,NaN,RU,NaN,Yes,NaN,Megahorn,Solar Beam,Earthquake,Iron Tail,NaN,High Horsepower,Earth Power,Energy Ball,NaN,NaN,NaN,Lunge,Shadow Ball,Throat Chop,NaN,Signal Beam,Sucker Punch,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Megahorn, Solar Beam, Earthquake, Iron Tail, H..."


In [ ]:
#the useful attributes column is at the end, let me move it ahead
column_to_move=final.pop('Useful_attributes')

insert_index=(final.columns.get_loc("Trash"))+1   #we want to insert this after trash

final.insert(insert_index,"Useful_Attributes",column_to_move)
final

,MonType1,MonType2,MonType3,Tier,In Wishlist,Used,Trash,Useful_Attributes,Move1,Move2,Move3,Move4,Move5,Move6,Move7,Move8,Move9,Move10,Move11,Move12,Move13,Move14,Move15,Move16,Move17,...,Move19,Move20,Move21,Move22,Move23,Move24,Move25,Move26,Move27,Move28,Move29,Move30,Move31,Move32,Move33,Move34,Move35,Move36,Move37,Move38,Move38.1,Move40,Move41,Move42,Move43
Name,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Abomasnow,Grass,Ice,NaN,RU,NaN,NaN,NaN,"Grass, Ice, Earthquake, Earth Power, Iron Tail...",NaN,Earthquake,Earth Power,Iron Tail,NaN,Rock Slide,Shadow Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Accelgor,Bug,NaN,NaN,RU,NaN,NaN,NaN,"Bug, Energy Ball, Sludge Bomb, Water Shuriken,...",NaN,Energy Ball,Sludge Bomb,Water Shuriken,Knock Off,NaN,NaN,NaN,Venoshock,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aerodactyl,Rock,NaN,NaN,RU,NaN,NaN,NaN,"Rock, Aqua Tail, Crunch, Earthquake, Ice Fang,...",Aqua Tail,Crunch,NaN,Earthquake,NaN,Ice Fang,Iron Head,NaN,NaN,NaN,Earth Power,NaN,NaN,NaN,Payback,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aggron,Steel,Rock,NaN,RU,NaN,Yes,NaN,"Steel, Rock, Aqua Tail, Avalanche, Crunch, Ear...",Aqua Tail,NaN,Avalanche,Crunch,NaN,Earthquake,NaN,Ice Punch,NaN,NaN,Shadow Claw,NaN,NaN,Payback,NaN,Dark Pulse,Earth Power,...,NaN,Ice Beam,NaN,Surf,NaN,NaN,NaN,Payback,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Ambipom,NaN,NaN,NaN,RU,NaN,NaN,NaN,"Foul Play, Gunk Shot, Ice Punch, Knock Off, Se...",NaN,NaN,NaN,Foul Play,Gunk Shot,Ice Punch,Knock Off,Seed Bomb,Shadow Claw,NaN,Triple Axel,Uturn,NaN,Iron Tail,Fury Cutter,Trailblaze,Smack Down,...,NaN,Chilling Water,NaN,Grass Knot,NaN,Shadow Ball,Throat Chop,NaN,NaN,NaN,NaN,Payback,NaN,NaN,Smack Down,NaN,NaN,NaN,Beat Up,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wigglytuff,NaN,NaN,NaN,RU,NaN,NaN,NaN,"Gyro Ball, Grass Knot, Solar Beam, Energy Ball...",NaN,Gyro Ball,Grass Knot,Solar Beam,NaN,NaN,Energy Ball,NaN,Ice Beam,NaN,NaN,NaN,NaN,NaN,Dark Pulse,NaN,Ice Spinner,...,Shadow Ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Ice Punch,NaN,Knock Off,Bubble Beam,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wishiwashi,Water,NaN,NaN,RU,NaN,NaN,NaN,"Water, Earthquake, Iron Tail, Ice Beam, Uturn",Earthquake,Iron Tail,Ice Beam,Uturn,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Wyrdeer,NaN,NaN,NaN,RU,NaN,Yes,NaN,"Megahorn, Solar Beam, Earthquake, Iron Tail, H...",Megahorn,Solar Beam,Earthquake,Iron Tail,NaN,High Horsepower,Earth Power,Energy Ball,NaN,NaN,NaN,Lunge,Shadow Ball,Throat Chop,NaN,Signal Beam,Sucker Punch,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


please try to sort the output and group it by the type, so that all dark moves are together, all fighting moves are together, etc.
(maybe you can also have a priority list for the moves of each type, and then display the moves in the order of the priority list)

In [ ]:
output_excel=final.loc[:,'MonType1':'Useful_Attributes']
output_excel.to_csv('Best_Mons_Versus_The_Elite.csv')

In [ ]:
'''
#champ team is currently a mono Fire team

elite_5='Fire'
mask_5 = coverage.map(lambda cell: elite_5 in cell if isinstance(cell, list) else False)

chosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1) & mask_5.any(axis=1))
final =clean_df[(mask_1 | mask_2 | mask_3 | mask_4 | mask_5)].loc[chosen_mons]

final['Useful_attributes'] = final[final.columns].apply(lambda row: ', '.join(str(item) for item in row if pd.notna(item)),axis=1)

column_to_move=final.pop('Useful_attributes')
insert_index=(final.columns.get_loc("Trash"))+1   #we want to insert this after trash
final.insert(insert_index,"Useful_Attributes",column_to_move)
final=final.loc[:,'MonType1':'Useful_Attributes']
final=final.drop(columns=['Tier','In Wishlist','Used','Trash'])
'''

'\n#champ team is currently a mono Fire team\n\nelite_5=\'Fire\'\nmask_5 = coverage.map(lambda cell: elite_5 in cell if isinstance(cell, list) else False)\n\nchosen_mons=(mask_1.any(axis=1) & mask_2.any(axis=1) & mask_3.any(axis=1) & mask_4.any(axis=1) & mask_5.any(axis=1))\nfinal =clean_df[(mask_1 | mask_2 | mask_3 | mask_4 | mask_5)].loc[chosen_mons]\n\nfinal[\'Useful_attributes\'] = final[final.columns].apply(lambda row: \', \'.join(str(item) for item in row if pd.notna(item)),axis=1)\n\ncolumn_to_move=final.pop(\'Useful_attributes\')\ninsert_index=(final.columns.get_loc("Trash"))+1   #we want to insert this after trash\nfinal.insert(insert_index,"Useful_Attributes",column_to_move)\nfinal=final.loc[:,\'MonType1\':\'Useful_Attributes\']\nfinal=final.drop(columns=[\'Tier\',\'In Wishlist\',\'Used\',\'Trash\'])\n'

In [ ]:
#something I was cooking above, idk what I was cooking
#ah, I was rearranging it to be more readable, I gotta make sure to implement this in the regular code too

'''
final['Useful_attributes'] = final[final.columns].apply(lambda row: ', '.join(str(item) for item in row if pd.notna(item)),axis=1)

column_to_move=final.pop('Useful_attributes')
insert_index=(final.columns.get_loc("Trash"))+1   #we want to insert this after trash
final.insert(insert_index,"Useful_Attributes",column_to_move)
final=final.loc[:,'MonType1':'Useful_Attributes']
'''

'\nfinal[\'Useful_attributes\'] = final[final.columns].apply(lambda row: \', \'.join(str(item) for item in row if pd.notna(item)),axis=1)\n\ncolumn_to_move=final.pop(\'Useful_attributes\')\ninsert_index=(final.columns.get_loc("Trash"))+1   #we want to insert this after trash\nfinal.insert(insert_index,"Useful_Attributes",column_to_move)\nfinal=final.loc[:,\'MonType1\':\'Useful_Attributes\']\n'

In [ ]:
#final.to_csv('temp_mons_June.csv')

In [ ]:
#use df.query for stuff
#read all old comments
#separate this program into preprocessing and post processing, store an intermediate file

copy paste technician mons before and after all the move deletions so that they don't lose their moves
Also Strong Jaw mons and stuff ig